# All pairwise associations between Gene Expression and MicroRNA data in TCGA
Check out more notebooks at our ['Regulome Explorer Repository'](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer)!


```
Title:   All pairwise associations between Gene Expression and MicroRNA data in TCGA
Author:  Boris Aguilar
Created: 03-11-2020
Purpose: Demonstrate how to use the Regulome explorer functionality to find significant associations from Gene expression and microRNA BigQuery tables
URL:     https://github.com/isb-cgc/Community-Notebooks/blob/master/Correlation-GeneExpression-MicroRNA


```
***

In this notebook, we use BigQuery to find significant correlations between all possible pairs of genes in the RNAseq data and unique identifiers in the microRNA data available in TCGA. The cohort for this analysis consists of BrCA patients that are 50 years old or younger at the time of diagnosis and Stage IIA as pathological state.

We used Bigquery to compute Pearson correlations and identify significant correlation coefficients using a BigQuery [user-defined function](https://cloud.google.com/bigquery/user-defined-functions). The necessary queries are generated using python functions implemented in the [Regulome Explorer module](https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module). 

### Authentication
The first step is to authorize access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) and alternative authentication methods can be found [here](https://googleapis.github.io/google-cloud-python/latest/core/auth.html).

In [1]:
%matplotlib inline
from google.cloud import bigquery
from scipy import stats
import re_module.bq_functions as regulome
import numpy as np

### Parameters
In this experiment we will use Gene Expression data and microRNA data available in ISB-CGC. The information about the Bigquery tables and their columns are stored in the Features dictionary. The significance level and the minimum number of patients for the correlation analysis are free parameteres that can be changed.

If you want to test this notebook checking all possible pairs please replace "labels1 = ..." with "labels1 = ' IS NOT NULL '" but know the computations will take several minutes to complete, we recommend to generate the query and copy it in the [console](https://console.cloud.google.com/bigquery).

In [2]:
# information from Bigquery tables
Features = {     'Gene Expression' : { 'table'  : 'isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression',
                                       'symbol' : 'gene_name',
                                       'study'  : 'project_short_name',
                                       'data'   : 'AVG( LOG10( HTSeq__Counts + 1 ) ) ',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',  
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND HTSeq__Counts IS NOT NULL',
                                       'dattype': 'numeric' },
             'MicroRNA Expression': {  'table'  : 'isb-cgc.TCGA_hg38_data_v0.miRNAseq_Expression',
                                       'symbol' : 'mirna_id',
                                        'study' : 'project_short_name',
                                       'data'   : 'AVG( reads_per_million_miRNA_mapped )',
                                       'rnkdata': 'data',
                                       'avgdat' : 'avgdata',
                                   'patientcode': 'case_barcode',
                                    'samplecode': 'sample_barcode',
                                       'where'  : 'AND reads_per_million_miRNA_mapped IS NOT NULL',
                                       'dattype': 'numeric'}
                
               }

# parameteres for Gene expression data
feat1 = Features['Gene Expression']
cohort1 = feat1['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) "
labels1 = ' =  \'RNU7-6P\' '  #' IS NOT NULL ' use this for all pair-wise comparison; or Change RNU7-6P for your favorite gene 
#labels1 = ' IS NOT NULL '

# parameteres for Gene expression data
feat2 = Features['MicroRNA Expression'] 
cohort2 = feat2['patientcode'] + " IN ( SELECT case_barcode FROM cohort ) "
labels2 = ' IS NOT NULL '

# significance level
significance_level = 0.001 # only 0.05, 0.01, 0.005, and 0.001 are alowed !!

# minimun correlation
r_treshold = 0.3

# minimun number of samples
nsamples = 25

### Read cohort
The cohort for this analysis consists of BrCA patients that 50 years old or younger at the time of diagnosis and Stage IIA as pathological state. 

In [3]:
cohort = """
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis < 50
      AND pathologic_stage = 'Stage IIA'
)
"""

### Queries to read tables
The queries to retreive the necessary data, query_t1 for Gene expression and query_t2 for micro RNA, are generated using a function available in the regulome explorer module available in here https://github.com/isb-cgc/Community-Notebooks/tree/master/RegulomeExplorer/re_module.  

In [4]:
query_t1 = regulome.generic_numeric_bqtable ( 'table1' , feat1, cohort1, labels1 )
query_t2 = regulome.generic_numeric_bqtable ( 'table2' , feat2, cohort2, labels2 ) 

### Queries to join tables and perform statistics
The two tables are integrated into one table to compute correlations. The query to perform these operations is obtained by using a function implemented in the regulome explorer module.  'sql' contains the complete query to perform the correlations.  You can copy the query (which is printed ) to the Bigquery - Google cloud console to run the query and obtain the results. The computations can take 10 minutes. Alternatively you can go to "Run the Query" section of this notebooks. 

Note: `cgc-05-0042.functions.significance_level_ttest2` is a [persistent function](https://cloud.google.com/bigquery/docs/reference/standard-sql/user-defined-functions)  that estimates upper bounds of p-values ( significance levels ), so we can filter out correlations that are not statistically significant ( p-value > significance_level ). This function only work for N < 500. For N>500 we recommend the normal approximation of the t distribution.

In [5]:
summ_query = regulome.get_summarized_table('Gene Expression',feat1,'MicroRNA Expression',feat2)
f_pval = regulome.query_js_tscore_pvalue()

sql = (f_pval + 'WITH' + cohort + ',' + query_t1 + ',' + query_t2 + ',' + summ_query + """
SELECT symbol1, symbol2, n, correlation,
   tscore_to_p(t, n-2, 2) as p
FROM (
   SELECT *, 
         ABS(correlation)*SQRT( (n-2)/((1-correlation*correlation))) AS t
   FROM summ_table
   WHERE n > {0} AND ABS(correlation) >= {1} AND ABS(correlation) < 1.0
     )
WHERE  `cgc-05-0042.functions.significance_level_ttest2`(n-2, t) <= {2}
""".format( str(nsamples), str(r_treshold), str(significance_level) ) )

print( sql )  # This Query perform the computations

CREATE TEMPORARY FUNCTION tscore_to_p(a FLOAT64, b FLOAT64, c FLOAT64)
 RETURNS FLOAT64
 LANGUAGE js AS
"""
  return jStat.ttest(a,b,c); //jStat.ttest( tscore, n, sides)
"""
OPTIONS (
 library="gs://ba-cgc-bigquery/jstat/dist/jstat.min.js"
);

WITH
cohort AS(
SELECT case_barcode FROM `isb-cgc.TCGA_bioclin_v0.Clinical`
WHERE project_short_name = "TCGA-BRCA" AND age_at_diagnosis < 50
      AND pathologic_stage = 'Stage IIA'
)
,
table1 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SELECT
      gene_name AS symbol, 
      AVG( LOG10( HTSeq__Counts + 1 ) )  AS data,
      case_barcode AS ParticipantBarcode
   FROM `isb-cgc.TCGA_hg38_data_v0.RNAseq_Gene_Expression`
   WHERE case_barcode IN ( SELECT case_barcode FROM cohort )     # cohort 
         AND gene_name  =  'RNU7-6P'   # labels 
         AND HTSeq__Counts IS NOT NULL  
   GROUP BY
      ParticipantBarcode, symbol
   )
)
,
table2 AS (
SELECT
   symbol,
   data AS rnkdata,
   ParticipantBarcode
FROM (
   SE

### Run the Query
The following steps run BigQuery and generate a table with significant associations.
Note: alpha is an upper bound of the significant level. 

In [6]:
bqclient = bigquery.Client()
df_results = regulome.runQuery ( bqclient, sql, [], [], [], dryRun=False )
df_results[0:19]


 in runQuery ... 
    this query processed 22617633294 bytes 
    Approx. elpased time : 3899 miliseconds 


,symbol1,symbol2,n,correlation,p
0,RNU7-6P,hsa-mir-1915,85,0.806421,1.526383e-20
1,RNU7-6P,hsa-mir-6799,85,0.391481,2.131067e-04
2,RNU7-6P,hsa-mir-7641-1,85,0.754290,8.987106e-17
3,RNU7-6P,hsa-mir-548at,85,0.368848,5.166153e-04
4,RNU7-6P,hsa-mir-603,85,0.524061,2.710132e-07
5,RNU7-6P,hsa-mir-518f,85,0.582660,5.111541e-09
6,RNU7-6P,hsa-mir-4435-2,85,0.386184,2.636972e-04
7,RNU7-6P,hsa-mir-4525,85,0.635127,7.048246e-11
8,RNU7-6P,hsa-mir-6796,85,0.637084,5.913008e-11
9,RNU7-6P,hsa-mir-521-2,85,0.486841,2.349373e-06


Note that the nonsignificant correlations ( p > significance level ) were filtered out.
We can also use python to compute p values from correlations and number of patients:

In [7]:
if not df_results.empty:
    f = lambda n, correlation  : (1.0 - stats.t.cdf( abs(correlation) * np.sqrt( (n- 2.0) / (1.0 - correlation**2 )), n-2)) * 2.0  
    df_results['p-python'] = df_results.apply(lambda x: f(x.n,x.correlation), axis=1)

df_results[0:19]

,symbol1,symbol2,n,correlation,p,p-python
0,RNU7-6P,hsa-mir-1915,85,0.806421,1.526383e-20,0.000000e+00
1,RNU7-6P,hsa-mir-6799,85,0.391481,2.131067e-04,2.115077e-04
2,RNU7-6P,hsa-mir-7641-1,85,0.754290,8.987106e-17,0.000000e+00
3,RNU7-6P,hsa-mir-548at,85,0.368848,5.166153e-04,5.135466e-04
4,RNU7-6P,hsa-mir-603,85,0.524061,2.710132e-07,2.642967e-07
5,RNU7-6P,hsa-mir-518f,85,0.582660,5.111541e-09,4.911054e-09
6,RNU7-6P,hsa-mir-4435-2,85,0.386184,2.636972e-04,2.618221e-04
7,RNU7-6P,hsa-mir-4525,85,0.635127,7.048246e-11,6.641332e-11
8,RNU7-6P,hsa-mir-6796,85,0.637084,5.913008e-11,5.566814e-11
9,RNU7-6P,hsa-mir-521-2,85,0.486841,2.349373e-06,2.306686e-06
